<a href="https://colab.research.google.com/github/ojuba-org/arabic-ml-data/blob/master/tashkeel_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tensorflow documentation

* [v2](https://www.tensorflow.org/tutorials/text/text_generation)
* [v1](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/sequences/text_generation.ipynb)

In [1]:
%tensorflow_version 2.x

In [2]:
import sys
import os
import time

In [3]:
import re
import unicodedata

In [4]:
import numpy as np

In [5]:
import tensorflow as tf

In [ ]:
#tf.enable_eager_execution()

In [ ]:
#import tensorflow.compat.v1 as tf

In [6]:
print("tf version = {} and py version = {}".format(tf.__version__, sys.version))

tf version = 2.4.1 and py version = 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [7]:
assert sys.version_info.major == 3, 'please use python 3'
assert tf.test.gpu_device_name()!='', 'no GPU, please enable GPU'

In [ ]:
#from tensorflow.keras.layers.experimental import preprocessing


In [8]:
! curl -sSLO https://github.com/ojuba-org/arabic-ml-data/archive/refs/heads/master.zip

In [12]:
! unzip -q master.zip "arabic-ml-data-master/corpora/quran/*.txt"

In [14]:
! cat arabic-ml-data-master/corpora/quran/*.txt > data.txt; wc -l data.txt

6236 data.txt


In [15]:
lines=[l.strip() for l in open('data.txt', 'r') if l.strip()]
s=set()
for l in lines: s.update(l)

In [16]:
alpha=sorted(s)
for i in alpha: print(unicodedata.name(i), repr(i))


SPACE ' '
ARABIC LETTER HAMZA 'ء'
ARABIC LETTER ALEF WITH MADDA ABOVE 'آ'
ARABIC LETTER ALEF WITH HAMZA ABOVE 'أ'
ARABIC LETTER WAW WITH HAMZA ABOVE 'ؤ'
ARABIC LETTER ALEF WITH HAMZA BELOW 'إ'
ARABIC LETTER YEH WITH HAMZA ABOVE 'ئ'
ARABIC LETTER ALEF 'ا'
ARABIC LETTER BEH 'ب'
ARABIC LETTER TEH MARBUTA 'ة'
ARABIC LETTER TEH 'ت'
ARABIC LETTER THEH 'ث'
ARABIC LETTER JEEM 'ج'
ARABIC LETTER HAH 'ح'
ARABIC LETTER KHAH 'خ'
ARABIC LETTER DAL 'د'
ARABIC LETTER THAL 'ذ'
ARABIC LETTER REH 'ر'
ARABIC LETTER ZAIN 'ز'
ARABIC LETTER SEEN 'س'
ARABIC LETTER SHEEN 'ش'
ARABIC LETTER SAD 'ص'
ARABIC LETTER DAD 'ض'
ARABIC LETTER TAH 'ط'
ARABIC LETTER ZAH 'ظ'
ARABIC LETTER AIN 'ع'
ARABIC LETTER GHAIN 'غ'
ARABIC LETTER FEH 'ف'
ARABIC LETTER QAF 'ق'
ARABIC LETTER KAF 'ك'
ARABIC LETTER LAM 'ل'
ARABIC LETTER MEEM 'م'
ARABIC LETTER NOON 'ن'
ARABIC LETTER HEH 'ه'
ARABIC LETTER WAW 'و'
ARABIC LETTER ALEF MAKSURA 'ى'
ARABIC LETTER YEH 'ي'
ARABIC FATHATAN 'ً'
ARABIC DAMMATAN 'ٌ'
ARABIC KASRATAN 'ٍ'
ARABIC FATHA 'َ'
A

In [23]:
tashkeel_set = { ch for ch in alpha if unicodedata.category(ch)=='Mn' }

In [24]:
for i in sorted(tashkeel_set): print(unicodedata.name(i), repr(i))

ARABIC FATHATAN 'ً'
ARABIC DAMMATAN 'ٌ'
ARABIC KASRATAN 'ٍ'
ARABIC FATHA 'َ'
ARABIC DAMMA 'ُ'
ARABIC KASRA 'ِ'
ARABIC SHADDA 'ّ'
ARABIC SUKUN 'ْ'


In [57]:
line=lines[0]; line

'بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ'

In [58]:
for i in line: print(unicodedata.name(i), repr(i))

ARABIC LETTER BEH 'ب'
ARABIC KASRA 'ِ'
ARABIC LETTER SEEN 'س'
ARABIC SUKUN 'ْ'
ARABIC LETTER MEEM 'م'
ARABIC KASRA 'ِ'
SPACE ' '
ARABIC LETTER ALEF 'ا'
ARABIC LETTER LAM 'ل'
ARABIC LETTER LAM 'ل'
ARABIC SHADDA 'ّ'
ARABIC FATHA 'َ'
ARABIC LETTER HEH 'ه'
ARABIC KASRA 'ِ'
SPACE ' '
ARABIC LETTER ALEF 'ا'
ARABIC LETTER LAM 'ل'
ARABIC LETTER REH 'ر'
ARABIC SHADDA 'ّ'
ARABIC FATHA 'َ'
ARABIC LETTER HAH 'ح'
ARABIC SUKUN 'ْ'
ARABIC LETTER MEEM 'م'
ARABIC FATHA 'َ'
ARABIC LETTER NOON 'ن'
ARABIC KASRA 'ِ'
SPACE ' '
ARABIC LETTER ALEF 'ا'
ARABIC LETTER LAM 'ل'
ARABIC LETTER REH 'ر'
ARABIC SHADDA 'ّ'
ARABIC FATHA 'َ'
ARABIC LETTER HAH 'ح'
ARABIC KASRA 'ِ'
ARABIC LETTER YEH 'ي'
ARABIC LETTER MEEM 'م'
ARABIC KASRA 'ِ'


In [59]:
def tokenize_tashkeel(line):
  offs=[ ix for ix, ch in enumerate(line) if ch not in tashkeel_set ]
  offs.append(len(line))
  return [ line[off: offs[ix+1]] for ix, off in enumerate(offs[:-1]) ]

In [60]:
print(len(line))
tokens=tokenize_tashkeel(line)
print(tokens)
[ len(token) for token in tokens]

37
['بِ', 'سْ', 'مِ', ' ', 'ا', 'ل', 'لَّ', 'هِ', ' ', 'ا', 'ل', 'رَّ', 'حْ', 'مَ', 'نِ', ' ', 'ا', 'ل', 'رَّ', 'حِ', 'ي', 'مِ']


[2, 2, 2, 1, 1, 1, 3, 2, 1, 1, 1, 3, 2, 2, 2, 1, 1, 1, 3, 2, 1, 2]

'ARABIC KASRA'